In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.optimize import minimize, least_squares
from torch.autograd.functional import hessian

# Load test samples
from aeons.tools import pickle_in
samples_g_1, samples_c_1, samples_w_1 = pickle_in("../test_samples/samples_1.pickle")

from aeons.covariance import logX_mu, logX_Sigmainv, points_at_iteration, X_mu, X_Sigma, X_Sigmainv, data_at_iteration
from aeons.true_distribution import generate_Xs
from aeons.lm_partial import analytic_lm_params
from aeons.bayes import logPr_bayes, logPr_laplace, logPr_gaussian, minimise_ls, minimise_bayes, minimise_gaussian
from aeons.likelihoods import likelihood, linear_like, quad_like, log_like, simple_like, middle_like, full_like
linear, quad, log = linear_like(), quad_like(), log_like()
simple, middle, full = simple_like(), middle_like(), full_like()
from aeons.hessian import hess_autograd

In [20]:
def logZ(logPmax, H, D, details=False):
    if details:
        print(f"logPr_max: {logPmax}, Hessian: {- 1/2 * np.log(abs(np.linalg.det(H)))}")
    return logPmax - 1/2 * np.log(abs(np.linalg.det(H))) + D/2 * np.log(2*np.pi)

In [ ]:
class Correlated:
    def __init__(self, y, likelihood, mean, covinv):
        self.y = y
        self.likelihood = likelihood
        self.mean = mean
        self.covinv = covinv
        self.N = len(y)

In [35]:
class Model:
    def __init__(self, y, likelihood, mean):
        self.y = y
        self.likelihood = likelihood
        self.mean = mean
        self.N = len(y)

    def minimise(self, x0, full_solution=False):
        def func(theta):
            return - self.logPr(theta)
        solution = minimize(func, x0)
        if full_solution:
            return solution
        else:
            return solution.x
    
    def logZ(self, theta_max, details=False):
        logPr_max = self.logPr(theta_max)
        H = self.hess(theta_max)
        D = len(theta_max) + 1
        return logZ(logPr_max, H, D, details)
    

In [ ]:
class LS(Model):
    def __init__(self, y, likelihood, mean):
        super().__init__()
    
    def L_sq(self, theta):
        loss = self.mean - self.likelihood.inverse(self.y, theta)
        return np.sum(loss**2)
    
    def s(self, theta):
        return np.sqrt(self.L_sq(theta)/self.N)

    def logPr(self, theta):
        L_sq = self.L_sq(theta)
        s = np.sqrt(L_sq/self.N)
        return -1/2 * self.N * np.log(2*np.pi*s**2) - L_sq/(2*s**2)
    
    def hess(self, theta_max):
        s = self.s(theta_max)
        y = torch.from_numpy(self.y)
        mean = torch.from_numpy(self.mean)
        theta_s_max = torch.tensor([*theta_max, s], requires_grad=True)
        def func(theta_s):
            if len(theta_s) == 2:
                theta, s = theta_s
            else:
                *theta, s = theta_s
            loss = mean - self.likelihood.inverse(y, theta, torch=True)
            L_sq = torch.sum(loss**2)
            return -1/2 * self.N * torch.log(2*torch.pi*s**2) - L_sq/(2*s**2)
        H = hessian(func, theta_s_max)
        return np.array(H)
    
    

In [31]:
points = 500
nk = 500 * np.ones(points)
mean_X = X_mu(nk)
covinv_X = X_Sigmainv(nk)
X = generate_Xs(nk)
theta_true = 10
y = simple.func(X, theta_true)

In [32]:
ls = LS(y, simple, mean_X)
theta_l = ls.minimise(theta_true)
ls.logZ(theta_l)

1684.335579042667

In [ ]:
class Model:
    def __init__(self, y, like, mean, covinv, model):
        self.model = model